In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/Othercomputers/My Laptop/Documents/Graduate_thesis/data/train'
test_dir = '/content/drive/Othercomputers/My Laptop/Documents/Graduate_thesis/data/test'
val_dir = '/content/drive/Othercomputers/My Laptop/Documents/Graduate_thesis/data/validation'

In [ ]:
model_dir = '/content/drive/Othercomputers/My Laptop/Documents/Graduate_thesis/models'